# CS 559 HW 4

## Question 1 [ 40 Points ]

**Support Vector Machines (SVMs)**

[25 points ] Download this dataset, split it as a 80% training and 20% test set. 


In [166]:
import pandas as pd
import numpy as np

from sklearn import model_selection

# the link to the Iris dataset in the hw doc didn't work,
# so I'll be importing through Scikit-learn directly
from sklearn.datasets import load_iris

In [164]:
iris = load_iris()

X = np.array(iris.data)
y = np.array(iris.target).reshape(-1,1)
iris_df = pd.DataFrame(data=np.column_stack([X, y]))

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

In [165]:
iris_df.head()

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [172]:
iris_df[4].unique() # how many classes?

array([0., 1., 2.])

(quickly doing feature scaling before any further modeling development)

In [167]:
from sklearn.preprocessing import StandardScaler

In [168]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Implement the support vector algorithm from scratch using Numpy and Pandas.

In [169]:
from sklearn import metrics

For the sake of simplicity, let's start by seeing if we can build a classifier that can at least to binary classification:

In [210]:
class BinarySVC:
    """
    Credit to this presentation by Patrick Loeber
    for helping me on the code: 
    https://www.youtube.com/watch?v=UX0f9BNBcsY
    """
    def __init__(self, learning_rate=0.001, lambda_param=0.01, epochs=1000):
        self.lr = learning_rate  # used for optimization
        self.regularizer = lambda_param  # used for regularization
        self.epochs = epochs
        self.w = None
        self.b = None

    def _linear_output(self, X):
        return np.dot(X, self.w) - self.b

    def fit(self, X_train, y_train):
        n_samples, n_features = X_train.shape

        binary_labels = np.where(y_train <= 0, -1, 1)

        # init the params of the model
        self.w = np.zeros(n_features)
        self.b = 0

        # using gradient descent
        for _ in range(self.epochs):
            for index, sample in enumerate(X_train):
                is_positive_class = binary_labels[index] * (self._linear_output(sample)) >= 1
                
                # derivatives used to update the weights
                if is_positive_class is True:
                    self.w -= self.lr * (2 * self.regularizer * self.w)
                else:  # belongs to the negative class
                    self.w -= self.lr * (
                        2 * self.regularizer * self.w - 
                        np.dot(sample.reshape(-1, 1), binary_labels[index])
                    )
                    self.b -= self.lr * binary_labels[index]

    def predict(self, X):
        """For the purpose of binary classification, we only output +1 or -1."""
        return np.sign(self._linear_output(X))


In [211]:
custom_model = BinarySVC()
custom_model.fit(X_train_scaled, y_train)
metrics.accuracy_score(y_test, custom_model.predict(X_test_scaled))

0.3

Great! At least the code works.
As we can see though, the accuracy is not great - this is an expected problem though, since only one of the labels in our actual dataset (0, 1, and 2) and can be outputted by our model implementation (which predicts only -1 and 1). 

Therefore, we'll have to do a little extra work to make this a proper, *multiclass* classifier. Let's go ahead and write an OVR classifier next (since that's the most popular scheme for multiclass SVCs):

In [216]:
class OneVersusRestSVC:
    def __init__(self, learning_rate=0.001, lambda_param=0.01, epochs=1000):
        self.lr = learning_rate  # used for optimization
        self.regularizer = lambda_param  # used for regularization
        self.epochs = epochs
        self.classifiers = dict()

    def fit(self, X_train, y_train):
        # A: for each class, train a classifier
        labels = np.unique(y_train)
        for positive_label in labels:
            # transform the n-1 "non-positive" labels to all be -1
            y_train_transformed = np.where(y_train != positive_label, -1, 1)
            binary_model = BinarySVC()
            binary_model.fit(X_train, y_train_transformed)
            # save this trained model for later predictions
            self.classifiers[positive_label] = binary_model

    def predict(self, X):
        # get all the models to "vote" on if the sample(s) fall in their positive class
        votes = dict()
        for pos_label, model in self.classifiers.items():
            votes[pos_label] = model._linear_output(X)

        # decide what final label ought to be
        y_pred_final = np.zeros(X.shape[0])

        for index in range(y_pred_final.shape[0]):
            # linear search for the most confident prediction
            strongest_pred_confidence, pos_label_of_strongest = -1, None
            for pos_label, predictions in votes.items():
                model_pred = predictions[index]
                if abs(model_pred) > strongest_pred_confidence:
                    strongest_pred = model_pred
                    pos_label_of_strongest = pos_label
             # now, we see if we can actually use that model's positive class label, or go to the others
            if model_pred > 0:
                y_pred_final[index] = pos_label_of_strongest
            else:  # model_pred < 0
                # need to take the positive label of one of the other classifiers
                candidate_classes = [
                    pos_label for pos_label in self.classifiers.keys()
                    if pos_label != pos_label_of_strongest
                ]
                # another search, narrowed down to JUST these classes
                labels_and_preds = [
                    (pos_label, predictions[index])
                    for pos_label, predictions in votes.items()
                    if pos_label != pos_label_of_strongest
                ]
                # take the max of these
                labels_and_preds.sort(reverse=True, key=lambda label_pred_tuple: label_pred_tuple[1])
                most_likely_label, _ = labels_and_preds[0]
                y_pred_final[index] = most_likely_label

        # all done!
        return y_pred_final

[10 points ] Report the accuracies for the train and test sets. Comment on whether your model has overfit.

In [219]:
custom_model2 = OneVersusRestSVC()
custom_model2.fit(X_train_scaled, y_train)

train_accuracy = metrics.accuracy_score(y_train, custom_model2.predict(X_train_scaled))
print(f"Train Accuracy of OVR Classifier: {round(train_accuracy * 100, 4)}%.")

test_accuracy = metrics.accuracy_score(y_test, custom_model2.predict(X_test_scaled))
print(f"Test Accuracy of OVR Classifier: {round(test_accuracy * 100, 4)}%.")

Train Accuracy of OVR Classifier: 88.3333%.
Test Accuracy of OVR Classifier: 90.0%.


Cowabunga! The accuracy of the OVR classifier is indeed much better than that of the initial binary support vector classifier above (probably because it is essentially an ensemble).

Also, it is hard to tell if the model has started overfitting. Although we do have a noticeable difference in the train and testing accuracies, it is less than 2% and the test accuracy is higher, which suggests that the model is still doing OK at generalizing to the test dataset. Overall, I would say the model is not yet overfit (although if I had more time I would definitely like to investigate this further using K-fold cross-validation).

[5 points] Test your model performance with the scikit-learn model. Comment on the difference in accuracy. 

In [220]:
from sklearn.svm import SVC

In [230]:
sklearn_svc = SVC(
    decision_function_shape='ovr',
    kernel='linear',  # because we're also using a linear model
    max_iter=1000,
    C=1.0,  # left at the default value
    random_state=0   
)
sklearn_svc.fit(X_train_scaled, np.squeeze(y_train))

train_accuracy = metrics.accuracy_score(y_train, sklearn_svc.predict(X_train_scaled))
print(f"Train Accuracy of Scikit-learn Classifier: {round(train_accuracy * 100, 4)}%.")

test_accuracy = metrics.accuracy_score(y_test, sklearn_svc.predict(X_test_scaled))
print(f"Test Accuracy of Scikit-learn Classifier: {round(test_accuracy * 100, 4)}%.")


Train Accuracy of Scikit-learn Classifier: 98.3333%.
Test Accuracy of Scikit-learn Classifier: 96.6667%.


Outrageous! So it appears that using an OVR classifier from Scikit-learn, we achieve noticeably higher accuracies on both train/test data. In my opinion, one likely explanation for this difference is because the Scikit-learn model is based off `libsvm`, which is capable of using the dual representation approach to solve SVC problems. This is very cool to see. Also, the `sklearn` model also has a more expected result, of having a higher accuracy on the training data than for the test data. It is still not that huge though, so I would say it hasn't quite overfitted the dataset just yet (but it's probably very close to).

## Question 2 [ 40 Points ]

**Decision Trees**

a. [5 points] Complete the `test_split` function.

In [130]:
from sklearn.datasets import load_iris
import numpy as np

In [131]:
# Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset):
	left, right = list(), list()
	for row in dataset:
		# move all nodes that are true to the left, false to the right
		if row[index] >= value:
			left.append(row)
		else:  # row[index] < value
			right.append(row)
	return left, right

b. [5 points] Complete the `gini_index` function.

In [132]:
# Calculate the Gini index for a split dataset
def gini_index(groups, classes):
	"""
	Using the calculation approach described here by Josh Gordon:
	https://youtu.be/LDRbO9a6XPU?t=354
	"""
	# count all samples at split point
	left_group, right_group = groups
	n_instances = len(left_group) + len(right_group)

	# sum weighted Gini index for each group
	gini = 0.0
	for group in groups:
		if len(group) > 0:  # protect against ZeroDivisionError
			num_classes_in_group = len(set(row[-1] for row in group))
			uncertainty = 1 - (1 / num_classes_in_group)
			gini += (len(group) / n_instances) * uncertainty
	return gini


c. [5 points] Complete the `get_split` function.

In [133]:
def get_split(dataset):
	"""Find the best split point by iterating over every feature / value
    and calculating the information gain."""
	unique_labels = list(set(row[-1] for row in dataset))
	b_index, b_value, b_score, b_groups = 999, 999, 999, None

	for feature_index in range(len(dataset[0])-1):
		for row in dataset:
			row_val = row[feature_index]
			groups = test_split(feature_index, row[feature_index], dataset) 
			gini = gini_index(groups, unique_labels)
			if gini < b_score:
				b_index, b_value, b_score, b_groups = (
					feature_index, row_val, gini, groups
				)
        
	return {'index': b_index, 'value': b_value, 'groups': b_groups}

In [134]:
# Create a terminal node value
def to_terminal(group):
	outcomes = [row[-1] for row in group]
	return max(set(outcomes), key=outcomes.count)

d. [15 points] Complete the `split` function.

In [147]:
# Create child splits for a node or make terminal
#Hint : Just call the to_terminal and get_split functions defined above. 

def split(node, max_depth, min_size, depth):
	left, right = node['groups']
	del(node['groups'])
 
	# check for a no split
	if not left or not right:
		node['left'] = node['right'] = to_terminal(left)
		return
	# check for max depth
	if depth >= max_depth:
		node['left'], node['right'] = (
			to_terminal(left), 
			to_terminal(right)
		)
		return

	# process left child
	if len(left) <= min_size:
		node['left'] = to_terminal(left)
	else:
		node['left'] = get_split(right)
		split(node['left'], max_depth, min_size, depth+1)
  
	# process right child
	if len(right) <= min_size:
		node['right'] = to_terminal(right)
	else:
		node['right'] = get_split(right)
		split(node['right'], max_depth, min_size, depth+1)

In [148]:
# Build a decision tree
def build_tree(train, max_depth, min_size):
	root = get_split(train)
	split(root, max_depth, min_size, 1)
	return root

e. [10 points] Print the tree. 

In [149]:
# Print a decision tree
def print_tree(node, depth=0):
	if isinstance(node, dict):
		print('%s[X%d < %.3f]' % ((depth*' ', (node['index']+1), node['value'])))
		print_tree(node['left'], depth+1)
		print_tree(node['right'], depth+1)
	else:
		print('%s[%s]' % ((depth*' ', node)))

In [150]:
iris = load_iris()

X = np.array(iris.data)
y = np.array(iris.target).reshape(-1,1)

data = np.append(X,y,axis=1)

In [155]:
tree = build_tree(data, 1, 1)
print_tree(tree)

[X3 < 3.000]
 [1.0]
 [0.0]


## Question 3 [ 20 Points ]

**Random Forests and Boosting**


### Loading Data

In [13]:
import pandas as pd

In [14]:
col_names = [
    "buying", "maint", "doors",
    "persons", "lug_boot", "safety"
]

In [16]:
car_df = pd.read_csv(
    './car/car.data',
    sep=",", usecols=list(range(0, 6)), names=col_names,
)

In [17]:
car_df.head()

,buying,maint,doors,persons,lug_boot,safety
0,vhigh,vhigh,2,2,small,low
1,vhigh,vhigh,2,2,small,med
2,vhigh,vhigh,2,2,small,high
3,vhigh,vhigh,2,2,med,low
4,vhigh,vhigh,2,2,med,med


In [18]:
features = col_names[:-1]
target = [col_names[-1]]

### Preprocessing & Splitting Data 

The approach I'll go with here is that because all the features are *technically* categorical, I'll one-hot encode them:

In [20]:
from sklearn import metrics, model_selection
from sklearn.preprocessing import OneHotEncoder

In [21]:
encoded_data = dict()
enc = OneHotEncoder(handle_unknown='ignore')
for feat in features:
    input_feat = car_df[feat].values.reshape(-1, 1)
    encoded_feat = enc.fit_transform(input_feat).toarray()
    encoded_data[feat] = encoded_feat

X = np.column_stack(list(
    encoded_data.values()
))

For the target column, I'll merely use sparse integer encoding:

In [30]:
classes = car_df["safety"].unique().tolist()
convert_to_int = lambda label: classes.index(label)

In [36]:
y = car_df["safety"].transform(convert_to_int)

In [38]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

### Training and Comparing Ensembles

First model - Random Forest coming right up:

In [48]:
from sklearn.ensemble import RandomForestClassifier

In [56]:
rf_clf = RandomForestClassifier(
    n_estimators=100, max_depth=1, 
    random_state=0, oob_score=True).fit(X_train, y_train)

y_pred = rf_clf.predict(X_test)
test_accuracy = metrics.accuracy_score(y_test, y_pred)
print(f"Random Forest Train Accuracy: {round(rf_clf.oob_score * 100, 4)}%")
print(f"Random Forest Test Accuracy: {round(test_accuracy * 100, 4)}%")


Random Forest Train Accuracy: 100%
Random Forest Test Accuracy: 28.3237%


Second model: Gradient Boosting, anyone?

In [39]:
from sklearn.ensemble import GradientBoostingClassifier

In [63]:
grad_boost_clf = GradientBoostingClassifier(
    n_estimators=100, learning_rate=1.0,
    max_depth=1, random_state=0).fit(X_train, y_train)

# get preds so we can eval accuracy
y_pred = grad_boost_clf.predict(X_train)
train_accuracy = metrics.accuracy_score(y_train, y_pred)
print(f"Gradient Boosted Classifier Train Accuracy: {round(train_accuracy * 100, 4)}%")

y_pred = grad_boost_clf.predict(X_test)
test_accuracy = metrics.accuracy_score(y_test, y_pred)
print(f"Gradient Boosted Classifier Test Accuracy: {round(test_accuracy * 100, 4)}%")



Gradient Boosted Classifier Train Accuracy: 36.5412%
Gradient Boosted Classifier Test Accuracy: 20.5202%


As we can see from above, the Random Forest classifier outperforms the Gradient Boosted by ~8% on the test dataset, when both were given the same 5 one-hot encoded features and hyperparameters for `n_estimators` and `max_depth`.

Both model have higher train accuracies than their respective test accuracies, which suggests both have started overfitting. However, it appears to be much more severe in the case of the Random Forest, which has a perfect score for training accuracy (100%), vs. the ~36.54% for the Gradient Boosted tree.